# Devices
This tutorial is aimed at explaining the entirety of writing Device drivers to interface with lab equipment. Let's take a look at the most basic Device, which is found in the Core tutorial:

In [7]:
from emergent.core import Device, Knob
from emergent.core.knob import knob

class TestDevice(Device):
    ''' Device driver for the virtual network in the 'basic' example. '''
    X = Knob('X')
        
    
dev = TestDevice('dev', hub=None, params={})

We've defined a single knob called X. We can interact with it like this:

In [8]:
dev.X = 2.3
print(dev.X)

2.3


## Device commands
So far, the knob is only a virtual object. In order to implement device control, we override the default setter method with whatever device command you want to send:

In [ ]:
from emergent.core import Device, Knob
from emergent.core.knob import knob

class TestDevice(Device):
    ''' Device driver for the virtual network in the 'basic' example. '''
    X = Knob('X')

    @X.command
    def X(self, x):
        print('Sending device command to update X to %f'%x)

    
dev = TestDevice('dev', hub=None, params={})
dev.X=4
print(dev.X)

Notice that the new setter method we defined doesn't explicitly redefine the variable - this is handled behind the scenes. You only need to define the device-specific command that you want to execute, and EMERGENT automatically generates the boilerplate to keep track of the virtual state.

## Device queries
The default Knob behavior is to store the last value defined by the user. This can lead to desynchronization between the virtual and physical values if someone turns a real knob in the lab. To remedy this, we can override the getter method through the "query" decorator. Here, we simulate sending a command to a device and getting a noisy version back:

In [ ]:
from emergent.core import Device, Knob
import numpy as np

class TestDevice(Device):
    ''' Device driver for the virtual network in the 'basic' example. '''
    X = Knob('X')
    
    @X.query
    def X(self):
        response = self._X + np.random.normal(scale=0.01)
        return response
    
    @X.command
    def X(self, x):
        print('Sending device command to update X to %f'%x)
        
dev = TestDevice('dev', hub=None, params={})
dev.X=3
print(dev.X)

# A word on properties
The original version of EMERGENT represented device states through dictionaries. This representation is still used to communicate between Devices and Hubs; for example, you can access the state like this:

In [ ]:
dev._state()

You can also set the state using the actuate() method:

In [ ]:
dev.actuate({'X':7})
print(dev.X)

The new property-based state representation has several advantages over the dict-based representation:
* Quality of life: setting the property directly is quicker to type than passing a dict into the actuate method. Knob declaration is also simpler and cleaner with this method.
* Better memory compartmentalization: updating individual properties instead of a state dict reduces memory overlap between knobs, which will be useful in future parallelization efforts. Note that memory is still shared by the Hub.
* Device synchronization: the previous dict-based method simply logged the last state sent to the device, which could become unsynchronized with the device if a physical knob was turned in the lab. Now, you can override the getter method to request the actual state from the device.

# Simulation mode
For offline device testing, any device can be put in simulation mode. In this mode, any commands and queries are bypassed and only the internal variable is updated and returned. To activate simulation mode, just set Device.simulation=True:

In [ ]:
print('Simulation mode off')
dev = TestDevice('dev', hub=None, params={})
dev.X=3
print(dev.X)

print('\nSimulation mode on')
dev = TestDevice('dev', hub=None, params={})
dev.simulation = True
dev.X=3
print(dev.X)

# Example driver: Toptica DLC Pro
In this example, we show how to write a device driver for the Toptica DLC Pro.

In [ ]:
from emergent.core import Device, Knob

class DLCPro(Device):
    piezo = Knob('piezo')
    current = Knob('current')
        
    def __init__(self, name, hub, addr = '169.254.120.100'):
        super().__init__(name, hub)
        self.addr = addr

    def _connect(self):
        self.client = self._open_tcpip(self.addr, 1998)
        for i in range(8):
            r=self.client.recv(4096)

    @piezo.command
    def piezo(self, V):
        self.client.sendall(b"(param-set! 'laser1:dl:pc:voltage-set %f)\n"%V)
        for i in range(3):
            self.client.recv(2)    

    @current.command
    def current(self, I):
        self.client.sendall(b"(param-set! 'laser1:dl:cc:current-set %f)\n"%I)
        for i in range(3):
            self.client.recv(2)
     
    @piezo.query
    def piezo(self):
        self.client.sendall(b"(param-ref 'laser1:dl:pc:voltage-set)\n")
        V = float(str(self.client.recv(4096), 'utf-8').split('\n')[0])
        for i in range(2):
            self.client.recv(2)
        return V
    
    @current.query
    def current(self):
        self.client.sendall(b"(param-ref 'laser1:dl:cc:current-set)\n")
        I = float(str(self.client.recv(4096), 'utf-8').split('\n')[0])
        for i in range(2):
            self.client.recv(2)
        return I
  
laser = DLCPro('laser', None)
laser._connect()

We can query the laser controller for the current setpoints just by accessing the class attributes:

In [ ]:
print(laser.piezo)
print(laser.current)

We can set the current and piezo voltage through the setter callbacks just by updating the class attributes:

In [ ]:
laser.piezo = 37.4
laser.current = 129.5

The power of the property-based approach should be clear - this framework allows us to manipulate physical devices just by changing virtual variables! Compared to a function-based approach, this implementation allows natural, human-readable interactions with devices.

# Example driver: Bristol 871 Wavemeter
In this example, we'll show the implementation of a Device for measurement only. Instead of Knobs, we'll add Sensors, which are basically just read-only versions.

In [ ]:
from emergent.core import Device, Sensor

class Bristol871(Device):
    frequency = Sensor('frequency')
    power = Sensor('power')

    def __init__(self, name, hub, addr = '10.199.199.1', port = 23):
        super().__init__(name, hub)
        self.addr = addr
        self.port = port

    def _connect(self):
        try:
            self.client = self._open_tcpip(self.addr, self.port)
            for i in range(2):
                self.client.recv(4096)
        except Exception as e:
            print(e)
            self._connected = 0
            return
        self._connected = 1

    def _query(self, msg, threshold = None):
        self.client.sendall(b'%s\n'%msg)
        resp = float(str(self.client.recv(1024), 'utf-8').split('\r')[0])

        return resp

    @frequency.query
    def frequency(self):
        f = self._query(b':READ:FREQ?')
        if f < 1e-3:          # detect read failures and return None
            return None
        else:
            return f * 1000   # return frequency in GHz
    
    @power.query
    def power(self):
        return self._query(b':READ:POWER?')


wm = Bristol871('wavemeter', None, addr='10.199.199.1', port=23)
wm._connect()

We can retrieve measurements from the wavemeter by accessing its Sensor attributes:

In [ ]:
print(wm.frequency)
print(wm.power)
wm.power=2

# Example driver: Mirrorcle PicoAmp


In [ ]:
import time
from emergent.core import Device, Knob, Sensor
import numpy as np
import sys
import os
char = {'nt': '\\', 'posix': '/'}[os.name]
sys.path.append(char.join(os.getcwd().split(char)[0:-1]))
import logging as log

class PicoAmp(Device):
    X = Knob('X')
    Y = Knob('Y')
    power = Sensor('power')
    ''' Device driver for the Mirrorcle PicoAmp board. '''
    def __init__(self, name, params = {'labjack': None, 'type': 'digital'}, hub = None):
        ''' Initialize the Device for use. '''
        super().__init__(name, hub = hub, params = params)
        self.addr = {'A': '000', 'B': '001', 'C': '010', 'D': '011', 'ALL': '111'}
        self.labjack = params['labjack']
        assert self.params['type'] in ['digital', 'analog']

    def _connect(self):
        ''' Initializes the PicoAmp via SPI. '''
        if self.labjack._connected:
            if self.params['type'] == 'digital':
                self.labjack.spi_initialize(mode=0, CLK = 0, CS = 1, MISO = 3, MOSI = 2)
            self.labjack.PWM(3, 49000, 50)

            if self.params['type'] == 'digital':
                FULL_RESET = '001010000000000000000001'    #2621441
                ENABLE_INTERNAL_REFERENCE =  '001110000000000000000001'     #3670017
                ENABLE_ALL_DAC_CHANNELS = '001000000000000000001111'      #2097167
                ENABLE_SOFTWARE_LDAC = '001100000000000000000001'    #3145728

                self.Vbias = 80.0
                for cmd in [FULL_RESET, ENABLE_INTERNAL_REFERENCE, ENABLE_ALL_DAC_CHANNELS, ENABLE_SOFTWARE_LDAC]:
                    self.command(cmd)
        else:
            log.error('Error: could not initialize PicoAmp - LabJack not connected!')

    @power.query
    def power(self):
        return self.labjack.AIn(0)
        
    @X.command
    def X(self, x):
        self.setDifferential(x, 'X')

    @Y.command
    def Y(self, y):
        self.setDifferential(y, 'Y')

    def command(self, cmd):
        ''' Separates the bitstring cmd into a series of bytes and sends them through the SPI. '''
        lst = []
        r = 0
        for i in [0, 8, 16]:
            lst.append(int(cmd[i:8+i],2))
        r = self.labjack.spi_write(lst)

    def digital(self, V):
        ''' Converts an analog voltage V to a 16-bit string for the DAC '''
        Range = 200.0
        Vdigital = V/Range * 65535

        return format(int(Vdigital), '016b')

    def setDifferential(self, V, axis):
        ''' Sets a target differential voltage V=HV_A-HV_B if axis is 'X' or V=HV_C-HV_D if axis is 'Y'.
            For example, if V=2 and  axis is 'X', this sets HV_A=81 and HV_2=79.
            Allowed range of V is -80 to 80.'''
        if self.params['type'] == 'digital':
            V = np.clip(float(V), -80, 80)
            cmdPlus = '00' + '011' + {'X':self.addr['A'], 'Y': self.addr['C']}[axis] + self.digital(self.Vbias+V)
            cmdMinus = '00' + '011' + {'X':self.addr['B'], 'Y': self.addr['D']}[axis] + self.digital(self.Vbias-V)
            self.command(cmdPlus)
            self.command(cmdMinus)
        else:
            V = np.clip(float(V),-5,5)
            channel = {'X':0, 'Y':1}[axis]
            self.labjack.AOut(channel, V, TDAC=True)


# Example driver: LabJack T7

In [24]:
import re

def AIN(name):
    ch = int(re.split('(\d+)',name)[1])
    def getter(self):
        _name = '_%s'%name
        return self.AIn(name[-1])

    def setter(self, newval):
        print("Sensor '%s.%s' value is read-only!"%(self.name, name))
    return knob(name, getter, setter)

def AOUT(name):
    ch = int(re.split('(\d+)',name)[1])
    _name = '_%s'%name
    def getter(self):
        return self.__dict__[_name]

    def setter(self, newval):
        self.__dict__[_name] = newval
        self.AOut(ch, newval)
    return knob(name, getter, setter)

def TDAC(name):
    ch = int(re.split('(\d+)',name)[1])
    _name = '_%s'%name
    def getter(self):
        return self.__dict__[_name]

    def setter(self, newval):
        self.__dict__[_name] = newval
        self.AOut(ch, newval, TDAC=True)
    return knob(name, getter, setter)

def DIN(name):
    ch = int(re.split('(\d+)',name)[1])
    def getter(self):
        _name = '_%s'%name
        return self.DIn(name[-1])

    def setter(self, newval):
        print("Sensor '%s.%s' value is read-only!"%(self.name, name))
    return knob(name, getter, setter)

def DOUT(name):
    ch = int(re.split('(\d+)',name)[1])
    _name = '_%s'%name
    def getter(self):
        return self.__dict__[_name]

    def setter(self, newval):
        self.__dict__[_name] = newval
        self.DOut(ch, newval)
    return knob(name, getter, setter)

In [30]:
# from labjack import ljm
import numpy as np
import logging as log


class LabJackDriver():
    ''' Python interface for the LabJack T7. '''
    simulation = False
    hub = None
    for i in range(2):
        ch = 'AIN%i'%i
        locals()[ch] = AIN(ch)

    for i in range(2):
        ch = 'AOUT%i'%i
        locals()[ch] = AOUT(ch)
    def __init__(self, params = {'device': 'ANY', 'connection': 'ANY', 'devid': 'ANY', 'arange': 10}):
        ''' Attempt to connect to a LabJack.

            Args:
                params (dict)
        '''
        self.params = params
        for param in ['device', 'connection', 'devid']:
            if param not in self.params:
                self.params[param] = 'ANY'
        if 'arange' not in self.params:
            self.params['arange'] = 10

        ''' Define a FIFO queue running in a separate thread so that multiple
            simultaneous threads can share a LabJack without interference. '''

#         self._connected = self.connect()

    def connect(self):
        try:
            self.handle = ljm.openS(self.params['device'],
                                    self.params['connection'],
                                    self.params['devid'])
            info = ljm.getHandleInfo(self.handle)

            self.deviceType = info[0]
            assert self.deviceType in [ljm.constants.dtT7, ljm.constants.dtT4]
            if self.deviceType == ljm.constants.dtT7:
                self._command('AIN_ALL_RANGE', self.params['arange'])
            self._command('AIN_ALL_NEGATIVE_CH', ljm.constants.GND)
            log.info('Connected to LabJack (%i).'%(info[2]))
            self.clock = 80e6       # internal clock frequency

            try:
                ''' Stop streaming if currently running '''
                ljm.eStreamStop(self.handle)
            except:
                pass

            return 1

        except Exception as e:
            log.error('Failed to connect to LabJack (%s): %s.'%(self.params['devid'], e))

    def _command(self, register, value):
        ''' Writes a value to a specified register.

            Args:
                register (str): a Modbus register on the LabJack.
                value: the value to write to the register.
                '''
        ljm.eWriteName(self.handle, register, value)

    def _write_array(self, registers, values):
        ljm.eWriteNames(self.handle, len(registers), registers, values)

    ''' Analog I/O '''
    def AIn(self, channel, num = 1):
        ''' Read a channel with optional averaging.

            Args:
                channel (int): number of the target AIN channel.
                num (int): number of measurements to perform and average.
        '''
        print('AIN')
        return
        vals = []
        for i in range(num):
            vals.append(ljm.eReadName(self.handle, 'AIN%i'%channel))
        return np.mean(vals)

    def AOut(self, channel, value, TDAC=False):
        ''' Output an analog voltage.

            Args:
                channel (int): number of the target DAC channel.
                value (float): Voltage in volts.
                TDAC (bool): If False, use a DAC channel (0-5 V); if True, use a TDAC channel with the LJTick-DAC accessory (+/-10 V).
        '''
        print('AOUT')
        return
        if not TDAC:
            self._command('%s%i'%('DAC', channel), value)
        else:
            self._command("TDAC%i"%channel, value)

    ''' Digital I/O '''
    def DIn(self, channel):
        return ljm.eReadName(self.handle, 'DIO%i'%channel)

    def DOut(self, channel, state):
        ''' Output a digital signal.

            Args:
                channel (str): a digital channel on the LabJack, e.g. 'FIO4'.
                state (int): 1 or 0
        '''
        if type(channel) is int:
            channel = 'FIO%i'%channel
        self._command(channel, state)

if __name__ == '__main__':
    lj = LabJackDriver(params={'devid': '470016970'})


AIN


In [39]:
def set_state(knob, target):
    knob = target
    print(knob)
set_state(dev.X, 3)
dev.X

3


2.3

SyntaxError: can't assign to literal (<ipython-input-38-29da790e286c>, line 1)